In [1]:
from IPython.display import display, HTML
display(HTML("<style>.container {width:100% !important;} </style>"))

In [2]:
import os
import datetime
import sys
import glob
import random
import pickle

import numpy         as np
import pandas        as pd

from scipy.fftpack   import fft
from pydub           import AudioSegment
from IPython.display import Audio

from pydub.silence   import split_on_silence
from pydub.silence   import detect_leading_silence

from tqdm            import tqdm

In [3]:
os.environ['PYTHONHASHSEED'] = str(42)
np.random.seed(42)
random.seed(42)

<h1 style="background-color:LightGreen;"> <center> <a id='start_cell'></a> Table Of Contents </center></h1>

[Create Datasets](#create_dataset) </br>

<h1 style="background-color:#3cA8EF;"> <center> <a id='create_dataset'></a> Choose Files </center> </h1>

<h1 style="color:red"> Choose speakers with at least 10 files </h1>

In [4]:
if sys.platform != "win32":
    # linux    
    dataset_path = r"/home/amitli/Datasets/CV_14/cv-corpus-14.0-2023-06-23-ru/cv-corpus-14.0-2023-06-23/ru/"                  
else:
    # windows    
    dataset_path = r"C:\Users\amitli\Datasets\cv-corpus-14.0-2023-06-23\ru"

In [5]:
CHOOSE_FILES = False
    
if CHOOSE_FILES is True:   
    train_df = pd.read_csv(dataset_path + "/train.tsv", delimiter="\t")
    test_df  = pd.read_csv(dataset_path + "/test.tsv",  delimiter="\t")
    df_all   = pd.concat([train_df, test_df])
    
    grouped         = df_all.groupby('client_id')
    filtered_groups = grouped.filter(lambda x: len(x) >= 10)
    df              = pd.DataFrame(filtered_groups)
    
    df.to_csv("cv_14_walkie_ru.csv")
else:
    df = pd.read_csv("cv_14_walkie_ru.csv")

In [6]:
print(f"Number of speakers: {len(set(df.client_id.values))}")
print(f"Dataframe size    : {len(df)}")

Number of speakers: 527
Dataframe size    : 29415


<h2> Choose that each speaker between 10 to 30 files: </h2>

In [7]:
grouped         = df.groupby('client_id')
filtered_groups = grouped.filter(lambda x: len(x) >= 10 and len(x) <= 30)
df              = pd.DataFrame(filtered_groups)
print(f"Number of speakers: {len(set(df.client_id.values))}")
print(f"Dataframe size    : {len(df)}")

Number of speakers: 260
Dataframe size    : 3314


<h1 style="background-color:#3cA8EF;"> <center> <a id='player'></a> Player </center> </h1>

In [8]:
from pygame import mixer
import time
import IPython
import os

pygame 2.1.0 (SDL 2.0.16, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [9]:
num_to_freq_file = {}

num_to_freq_file[0]  = 'encoded_tones/audiocheck.net_sin_500Hz_-3dBFS_1.5s.wav'
num_to_freq_file[1]  = 'encoded_tones/audiocheck.net_sin_1000Hz_-10dBFS_1s.wav'
num_to_freq_file[2]  = 'encoded_tones/audiocheck.net_sin_1100Hz_-10dBFS_1s.wav'
num_to_freq_file[3]  = 'encoded_tones/audiocheck.net_sin_1200Hz_-10dBFS_1s.wav'
num_to_freq_file[4]  = 'encoded_tones/audiocheck.net_sin_1300Hz_-10dBFS_1s.wav'
num_to_freq_file[5]  = 'encoded_tones/audiocheck.net_sin_1400Hz_-10dBFS_1s.wav'
num_to_freq_file[6]  = 'encoded_tones/audiocheck.net_sin_1500Hz_-10dBFS_1s.wav'
num_to_freq_file[7]  = 'encoded_tones/audiocheck.net_sin_1600Hz_-10dBFS_1s.wav'
num_to_freq_file[8]  = 'encoded_tones/audiocheck.net_sin_1700Hz_-10dBFS_1s.wav'
num_to_freq_file[9]  = 'encoded_tones/audiocheck.net_sin_1800Hz_-10dBFS_1s.wav'
num_to_freq_file[10] = 'encoded_tones/audiocheck.net_sin_1900Hz_-10dBFS_1s.wav'

for f in num_to_freq_file.values():
    if not os.path.exists(f):
        print(f"Missing encoding file: {f}")    

In [10]:
def play_file(filename, fileLengthInSeconds, log_file):
    mixer.music.load(filename)
    mixer.music.play()
    time.sleep(fileLengthInSeconds)    
    time.sleep(0.05)
    while mixer.music.get_busy():  
        str_log = "\n---> busy - wait -----\n"
        print(str_log)
        log_file.write(str_log)
        time.sleep(0.05)
    #time.sleep(0.05)

In [11]:
def play_beep(log_file, start_time):

    # log start
    end_time = time.time()
    str_log   = f"\nStart play beep, Elapse: {round(end_time-start_time, 3)}"        
    print(str_log)
    log_file.write(str_log)
    
    # play beep
    play_file(num_to_freq_file[1], 1, log_file)
    
    # log end
    end_time = time.time()
    str_log   = f"\nEnd play beep, Elapse: {round(end_time-start_time, 3)}"        
    print(str_log)
    log_file.write(str_log)                    

<h2 style="color:green"> Filter previous transmitted files: </h2>

In [12]:
client_filter_list = [
"0054c51dd99cb0ab5528e5111f68d0fbb8bd5fa25d85945c54bfc8c1d5b6d03023cac30abf6b152d9510cc1596e940d8338e5d0da9bdefc4c039b24f152bc6f4",
"0142be05b17b3b7ac16c17e7d0de8c248005a2e4b4b10a8509c619f67b354ad35df346a16ea2a0e15f9af4897490dbdd79ff87ad4633b73335b405e9f2ba73ea",
"0226170a76199e560e832eb685b38f08024c66edaa02e8a3b5ef21e094f27cc2da574f76418e362816f03c133b785334e3cd6e71be3e8b13e190a2997fc9a9f7",
"034644aeea75e969639c79c7c5d02ba9418db78ef098f85ad5d347e191a061c2947f8ccdce768529d2107c3e89f48c56d786a337cc1a3ff87253fe7371460014",
"047a5717cccf7800952fe0b3f2272153936bfffb599be575c5f039392e9e492f854e45282fead11b10e4596566ef1467c2e7fb85171642af677905aa923be5b0",
"059d5f2e926e0de282d9aad9df1d47ae2ff3afb4fb60f3c25edc2b1655635fd989356b6b0fcb2e1ceb11ea908f89a0d989005bbbb734c4e0e11de568dd5f1aeb",
"0639aa1aae541ec151ef9d5d312fa56037d9febfaf07f80ff9a842d4a0cf8cd3281905e0047373e073ff5fcb17a72c1ea7fba6119c3047efdb336b231949c30a",
"071cacc8279bc15667bb35d018aef67f19446d65f17e1f5e0e06328c26dfb512c8b7d495bf85a038e7775dc3db897df6cf7adcd86806e0553cfb25c3e5c8b206",
"0759fded105e11d07c4f8bf79b1a932787a31f383b911bf20aae9a716c2f8c1823405aeaf122ab784850c0b5b847ce791ba2a75bab56dc41a670ed3fc072ed5b",
"07d2ef37de064922c03a12daed75ffbef6eb256b20053a9a77778fa7828c0e4f3ff2b27c857b3af19f9ea1d28c44769804d88683aa3d9c1d5f338d286ca1bd9e",
"087ce1d4609572ccc7959ae5424ea046b564e9929d6bff4d558a0e0a55099ab579d03e0503833f8568b8b42c63968ec82a6ddd072ab367c3755c00822d6d6cc4",
"092dc4aa7926ac281bb25ce3c0df09481bf273afc7bdd920e60b7c5598f0829e68f3a3096b17adf823a2ffe6028cdea50f3436ba18c887c0cd4e096ae6198177",
"094a1d108d5104a3cea607d0ef3619e0b3e4457dd30e0b681014c111039183d20f49d2aaf12b360525c8472ecc4cfd3bb6f8ded05d3ed4501f1c1ec62603291c",
"0ad35178cdfda14fd1d15a73ba7d2cc253894394d1f5af084479bf1273c40e6d55a312bcad614a2c235768edb32ad357d89b9344f7581eb4b9f4ab5a19b58b61",
"0b6285eb6892db2ccf358f67e66cf7fff6048fb77852809879129ca6e27e24e1972544da31f3a5214c4430e25ae29ed1c86facff8bc39eb58206a073f05cfbbd",
"0cf5507fae1dc8d9bc7667cf11b4d0a74937787e4d63e2c8ed90eb4067eadc603952a58b41c429b8b8b6e01218ae2f06a8cff63f3946e92c7c4e34ba523a95c4",
"0e1f34acec7673f549d6c2a0804c9e31adf6417d1dd69776f915083483f070a5c9cdf784d155596f950a6991a5fe1a85cca6fd240218e981e108d203aece0230",
"106901773650fdc20bef015b2bf3736d34f06ebd4b3035281d394f53226354bea63c1b5034d2c23b0782d29f931a84e792fe38797ffc344ef71fd4ac92158b74",
"10e6b73937f6a97f74f9b71f6a6b471287acfb2df1c5ced57b1d75011877db476b3f85c318aa53124053c1508190982140bc0a1f9c959c7c4fae707080abff39",
"155ff986d69df026866a38fe6170148080ea087b972c40be92d762a31b7e9711fdc5d909c027e4441cbf113457d8653d4e5b94882e5e55a7e556ce7762ab404b",
"15bcfc292cad34525d29098e6437591365193f8a0fc3cf7e29651cc24dcf3c80c76d73d5bad07c577f4d10f86f30ae895d219e802cdc61d1a7f58c6839e350d9",
"15e21d7ffd2cabc565777cefc87e472a591a3c0eaeabff35df93b3de3ac5c3909a41fcefc021e77f66c4075cc0c2bdc593cea0970a442cb5ab12214ef2726f50",
"1697f86d95b70327ba8910a09fd61600dbb47aaf90f94b03b96f5829bee00f75a9e3f622ca1b6edc1ee0c5d7c3f5444bb1348fdf6a4199e6f241c42d59ffb6c2",
"16f95acad9d24ef878cdc13cc590699a50abdeeebe98a1fec8757a89694368567594105ca19d96816cb97d18d2e6ca94154616115c0b8e4703da69d72861f6fa",
"191c0c9030897284ea11972fb366b57340d63732f43df616477eb4e6d449d9cd3af364351718564e742c9c22ff3c1f7cf4b94310302c0bc44240bc379c1ac63a",
"1a54c889ce5c13cf3796af64f15cfeedfdf8eda0717f589f4fcb9b67411230e530850857575d1e1ed5a89fe27808cdf1c0b88cfc38779540671252c72f3c51df",
"1a82bb14e9805896e272110ace70de51e9a649391a5e29688027f9c170b3e88df906513ddb0a8df1c347cf7f99be9e5f516bd0273ff9f1620f5ed6b05ac8caf5",
"1b08242203a40a630e6331d379efcafc0d1a8503bb97f150955d893382f5e90063de7e4f4479f9ac7ffebb7bdb278dbef2c0b25c19aaa8b5d3d2a789d76660e0",
"1b2008563b27a5f4616e7b268b0a1300a718a394505e8b9f3453f5e2137086e448fd7873e7593c8a97bae9fae7d26d04626530dbe0924aaa8e47b45bb8ca5e2a",
"1bdae73931a36549707d6615ec09d1ef8d14c028e587d9285c7b5c8b31afe874cb2f7e8fca8cc157c1efc8ccc5585ace2c7e74fab389876e59c67338554f83e4",
"1d03fdd6a2a42a1c339b208868544fc01b76562a40c242400b63006830ee42a813e25e59bc6de5cecb453e41ff13219b1fcd1e5d44b2004ed1845cc21f7348ca",
"1df8e9a1131f574a639141b876b2cf5a03d4f2561ddf6ce64d9cd38e47df507967028ef6d68bd0ef0a91796f778e5f3135979e25032feec5cbec876a17e4799c",
"1e299f961c44249bd95adffd123bc5d308681b232c34069521922fa1b307c0cdd513b4efc9b31cf01f8b63a221b54b38a3ea0506cd707252d0cd97e6f5463cf8",
"217eec2a145c4b3e673f4f25aa1cb7459b69a8dd7476538062ee15a16f5e40f4c4c687a0ea4cf368ee002e65360c9495b239c7d99c6033c72e34ee32db3dbb44",
"22170fd9f6e7ae36ad23e310410a2730a1665464e6d47b7e40b2cc39736038537c4eadd1f743bdff3ce0924e0af9b031f6ca7e72e65e087e44721f0bbea48da7",
"22b3e5dfdeeeb767b08fed9a06a62cc5bfafe8219d9244aac862c7d95df385b556119400958d1ffb555f44a23d45843992d1348d8d506abd951c0b87511b3111",
"22d15a547fa491597b47b4bb52f2944873f2958bf08a687412a99fee383e36c2803555bc46f0bd89a1c94becd4b38fd36259574c3bb81ea4c34191df182ce8e7",
"2347fe3ca161592b98f9c6beb055e201fa7551d377eb92587cdc64a772b0078f72fbb4da546ed28de197acd22da5645f46bb58541a1c8119a7ff4d50c6ee94cd",
"23aba149f2585a5471984ae5eb8366791fdcc1d2ca00da5415708fba6507034afd45460cfeb09dc7ba9b09739905abaa1e81cfe6f6fc91023bc1e46f7e93ede4",
"2474e5e70bf2dd9a65158412647cc78719cc85b8dc6385a46feae9e5022fba1852179a6c5dae74770e979dfb78607c6f403b2c48fb2ff55c54997d6a8245286d",
"24b0e1b8e79963654f1f32fe0ff36c3f24ef9f163df4f8c842b834f337cff3093291790c0c93e60a00b34e26efc2b1e247d77a4ae514fb7d4b239d9681b48aa4",
"252e77041702e197c683e3613cd34ca50eea41a8c7e7e8077afbe9ddedd8b7cb9efb4225b31c49b5cfca140f5fcfddfad412f26224621004d4c86ab16f687c72",
"253f8b7db1f11ad028bb8639b5de6bdff30c5c43612a91ca03d9445ee462919ee66b3eff46a61ac690ca167da63a2ce14f932fd171ec85e518f38a9e26640b13",
"26fa55ee70dddc2db39a7f3fe5f7214a983eb989ab11a4752bd123dcc26c4d9917710c90e83eabb9721f0df50c88d1a9cd556d8b208ca4afb0b1a62bcd66339b",
"2758a01ad9dfe9ce4c685de47dad7833ec61593bc91839937d4ed46a02a8f97157c9ccd9c25b2f77f2ee4aaba4d15b0576b3bf7d7d3c6f49f1b4e15206e234ee",
"27df1223bbfbd89b36d61b8fed39e40a1460c3e2e5fa763ba7d3c1b0d305b2919d2c3bbed70fabe90ab8bd15848f507b2331e5f108dd1b1b8c06dfbe287282c5",
"2a5345c855de524be1866b92d0f60f711aa30bb9640897bd12044a1e915ad5775f03b7173d9cee37145ccfe2cc76bb4325fdd23b7b188d07aa3621a651acc51d",
"2b4d59ff468b90dcb94aa015e5a5d0b9cf95db002cca11524c0f22a18f1478ca0e1248332dc11edb7315b1ae9dce49fae2cb81b8bdf0aa2820e9f74566a69124",
"2bc2524e03d46d6cbf8d272d765fc461a52857e56d94f47a1be76d3ca46dc05b400ccd204932a9d3f5129ba25e7321376762fe0206e1a14256d40404338ba2b8",
"2d247a01a4a41c64e3df83af7f69dc048de0d85f17e0bbad82cb836a12341b606d1989ab278434272af1067b141f9f5b16ba1517798f4ad2efe345023a92ec93",
"2d56a34bd70db14be7b40cb02dd07d09e69dc950ca1d0d8ace656447ddfe3b32eb36bbc5f4116155afa7b4922c4aa227a6b660c00e61004c141ad9974ff5f9ae",
"2dc1577851c6b61465996f9d2a68cdca1531091fa04abb34f38f37d64d81101927bea0528d546ef1c10f5d98380629fc8b8fc4b0d128049da13cf06503ed5527",
"2fac57a3bd50217fc45ce8e25ecc4cabdfaddb4fd9468b8ab2ef76ca58f638bc28f4fa80afe33035f961c2591bf6b1eea9194b21c70396f0b3c01aa5defbd708",
"30a5a536328f1611a2ab1e978236d20e12e14363d6c589245a576644c3b78afbc97411b5eedbdfae075df4f621549f69878968c845bd734521aa472476d76f55",
"30b575036e06763f6e8061ffd13d55ea14f1d859edd878d6d82fb8d39c9ec0bc390f869a14acffe217a5ef856ecd149a79e7449d4af3bb3bdf4c89ce8a9e8bb6",
"30d24834494f6b3b5f9a6790d00380a12476f17fab1c0b7755dfd132f5e2aed87c77164dbd51dedde37d7e343597c53db33c450ae2eab742df08416fd1dc4b5f",
"3298199ddafd85790e80e178e482b1b8f4911777f4b12e1198113858e2cbd1bc0d1fdb6cc12318e5ba3700330ecb9e04c2957ffc6125913b77833247550c2801",
"32a1697a89d6131f790243eff7d696f53e11a3120cf997218fa577fdac35dba9ebb53607c446abaa00bab9a6caec59ed3863d4fa64277cfd8b5694f1f721271a",
"353834406353e7917dc5bfb32852bc58c3051509e867a5bc880f4e7d4858c593ffebb66c27515c20c91f636b0520623ef7101080499ba5e5d91cadcbf39c14a9",
"35e91a096c8c9ccc146a48ef4f431fe25c016c665d0d81fce545f3e1778f2fb4e6e15c40e45c77e3f95b77ab7b78e93c3a64172425623817063e619a58d21b0c",
"368e8b20f5d651dce11670d36ddcb5ff21ff67a305a90a373f278f9e24e3360f19f71d1de5aa521e109f2b81f6ac516b5f5261a9d92da279dd4e5cc20ff85272",
"369e435017fe99673003e740b52d6b1f8fea5473aa6dbbb510fccd74091e4372b4bac262be795064b60ebc2ebfdeded957281098c6ff1a2b09d74f8ae1fb962a",
"36ed021b114b11704fff8a4ea526ba434aa1dde069c53b1ac7073c0b923164fdc7e2ce4b054a18068c7b4e14072f54934d1fd54306be10afdf921aeb30fe9640",
"3742174ae31b78b11558814a7196a5eec0f489ad3e728a2a125dd1a60c809a770eeaa758499c014bf728437835b4ea9c031a3a9c838b3882b35ed28744768ef3",
"37b0cf86f60c4c35a3a8180f2a7216612930de0db3722be1c9e4e41ee2a2b9c6284593925ee1f56567c4195e899a099fb976bd9a0bdd7f5419bcd6cd59e6037b",
"3929f5c37008fbbc1c939c56f84d046344cc7f6fc04099785f27a511eab4a064c19ec77358f033f02a4408c00080935618cbc82378c25afd0073c54ce158ced1",
"398a2fecdc4062765b95c96829093f0cccbdfe0338b882afc62ba575193253ab74529c5d4deeca542475b8ea1df2a1b3de1ee278a7ed477a809ed3a72d6d042e",
"3a93b25ffcfd75f34c18446ad9ae2c7309b701f5398d77d7b7a755ecd470bcc5e371ecd191b8226687e1307ae22ee804b94bf9bf3385d23f793da77dd472c61d",
"3aac6b8ee313a2eb49e281d760a637c88c908cd7bebc496e8d64de99600bf5181b0be515dd663d60c2446501d87760ad67f8d006f9533db7981b9a492873feb7",
"3acd5d776227b74111af82d1414f7099d604c1e87847d65afde513c6fdbacbfb009838a159319259789758dc282acaea7a39fcf732c470c1ef3c31416962d7b2",
"3afe4f01f91a9be65ff064d701c9fa55f95b6e30a4bc4b77a712c4212958f7066f58f9256ee23e48ce3155e0e8045b28234cc13c473e48a82c178466eb72669e",
"3c85523ea1b8da8e3510332a5fc6acbbdddfab68b07b512c397d08be78876c144a7be10119093962d362d5f289fd13433451afb9138d6a9647a3989a1c07ce86",
"3cfcc99337cc9f2616a192cd14d52a9b00405235e82a66433704b8ef02223b41581d1e872380db768ef9e0a5b6da8df67041c9183abaaef8e10217464718d3da",
"3d3b1cce5285c294286961f0a87ddeaf3ce9f05f3a3f03be66017237d27e87bb7a2fd1487a7a5b783f13949873b00a16c9edb6005614e5044e3cbce3a32b34ab",
"3daced327a76b5e96c990e47edda81d780819dfcb73e112ce06cc7931cbedc2c0ff8c74a0b59908ad0ad836a8f596632b264022ed8138b84e6badced104e5992",
"40ff1ed43df698ca860462956522f4857496f403a18da5f91220c46f032fb28ef408b07f14007241fe256590d843622d6a351c535b7ad47d6d7cddfd7b55ed5f"]

In [13]:
f                       = open("Lab_Clients.txt", "r")
tmp_lines               = f.readlines()
lab_transmitted_clients = []

for line in tmp_lines:
        
    line = line.replace('\n', '')
    line = line.replace(',', '')
    line = line.replace(' ', '')
    line = line.replace('\'', '')
    lab_transmitted_clients.append(line)
    
for val in client_filter_list:
    lab_transmitted_clients.append(val)



In [ ]:
def play_language(df, dataset_path,  start_from_client=0):
        
    current_date = datetime.datetime.now().strftime('%d_%m_%Y_%H_%M_%S')        
    all_clients  = list(set(df.client_id.values))
    all_clients.sort()

    mixer.init()     
    start_time = time.time()        
    with open(f"log_{current_date}.txt", "w") as f:
        
        play_beep(f, start_time)
        time.sleep(0.05)  
        play_beep(f, start_time)
        time.sleep(0.05)  
        play_beep(f, start_time)
        time.sleep(0.05) 
        
        for c_i in range(len(all_clients)):  
            
            if c_i < start_from_client:
                continue
                
            
            client_id    = all_clients[c_i]
            df_client    = df[df.client_id == client_id]            
            client_files = df_client.path.values
            
            if client_id in lab_transmitted_clients:
                print(f"client: {client_id} in filter (old trans list)")
                continue
            
                                                            
            str_log   = f"\n\nClient_Id: {client_id} [{c_i+1}/{len(all_clients)}] Files: {len(client_files)}"        
            print(str_log)
            f.write(str_log)
            
            # --- client beeps
            play_beep(f, start_time)
            time.sleep(0.05)  
            play_beep(f, start_time)
            time.sleep(0.05) 
                          
            for file_i in range(len(client_files)):      
                
                # get file params
                file         = client_files[file_i]
                fullFilePath = f"{dataset_path}/clips/{file}"
                fileLength   = AudioSegment.from_mp3(fullFilePath).duration_seconds

                # play beep
                play_beep(f, start_time)
                time.sleep(0.05)  
                
                # play file
                end_time = time.time()
                str_log = f"\n\t[{file_i+1}/{len(client_files)}] Start Play file: {file}, (length: {fileLength} seconds), Elapse: {round(end_time-start_time, 3)}"
                print(str_log)
                f.write(str_log)
                                
                play_file(fullFilePath, fileLength, f)                
                
                end_start = time.time()
                str_log = f"\n\t[{file_i+1}/{len(client_files)}] ---> Played file: {file}, length: {fileLength} seconds, Elapse: {round(end_start-start_time, 3)}"
                print(str_log)
                f.write(str_log)
                
                # sleep (prevent blocking)
                time.sleep(2)  
                              
            
        end_time = time.time()        
        str_log = f"\nClose File, Elapse: {round(end_time-start_time, 3)}"
        print(str_log)
        f.write(str_log)

        f.close()
        

In [ ]:
#AudioSegment.from_mp3(r"C:\Users\amitli\Datasets\cv-corpus-14.0-2023-06-23\ru\clips\common_voice_ru_21107819.mp3")
#AudioSegment.from_mp3(r"C:\Users\amitli\Datasets\cv-corpus-14.0-2023-06-23\ru\clips\common_voice_ru_21107820.mp3")

In [14]:
all_clients      = list(set(df.client_id.values))
df_tmp_fillterd  = df[~df.client_id.isin(lab_transmitted_clients)]
print(f"Number of clients             : {len(all_clients)}")
print(f"Number filter clients         : {len(lab_transmitted_clients)}")
print(f"After filter Number of clients: {len(set(df_tmp_fillterd.client_id.values))}")

Number of clients             : 260
Number filter clients         : 203
After filter Number of clients: 132


<h2 style="color:green"> Play: </h2>

In [ ]:
play_language(df, dataset_path, start_from_client=1)        

<h1 style="background-color:lightgreen;"> <center> <a id='player'></a> Player by cllients list </center> </h1>

In [15]:
def play_clients(df, dataset_path,  l_clientList):
    
    # fillter clients tpo send
    df = df[df.client_id.isin(l_clients_to_send)]
   
    # start ...
    current_date = datetime.datetime.now().strftime('%d_%m_%Y_%H_%M_%S')        
    all_clients  = list(set(df.client_id.values))
    all_clients.sort()

    mixer.init()     
    start_time = time.time()        
    with open(f"log_{current_date}.txt", "w") as f:
        
        play_beep(f, start_time)
        time.sleep(0.05)  
        play_beep(f, start_time)
        time.sleep(0.05)  
        play_beep(f, start_time)
        time.sleep(0.05) 
        
        for c_i in range(len(all_clients)):  
           
            
            client_id    = all_clients[c_i]
            df_client    = df[df.client_id == client_id]            
            client_files = df_client.path.values
           
                                                            
            str_log   = f"\n\nClient_Id: {client_id} [{c_i+1}/{len(all_clients)}] Files: {len(client_files)}"        
            print(str_log)
            f.write(str_log)
            
            # --- client beeps
            play_beep(f, start_time)
            time.sleep(0.05)  
            play_beep(f, start_time)
            time.sleep(0.05) 
                          
            for file_i in range(len(client_files)):      
                
                # get file params
                file         = client_files[file_i]
                fullFilePath = f"{dataset_path}/clips/{file}"
                fileLength   = AudioSegment.from_mp3(fullFilePath).duration_seconds

                # play beep
                play_beep(f, start_time)
                time.sleep(0.05)  
                
                # play file
                end_time = time.time()
                str_log = f"\n\t[{file_i+1}/{len(client_files)}] Start Play file: {file}, (length: {fileLength} seconds), Elapse: {round(end_time-start_time, 3)}"
                print(str_log)
                f.write(str_log)
                                
                play_file(fullFilePath, fileLength, f)                
                
                end_start = time.time()
                str_log = f"\n\t[{file_i+1}/{len(client_files)}] ---> Played file: {file}, length: {fileLength} seconds, Elapse: {round(end_start-start_time, 3)}"
                print(str_log)
                f.write(str_log)
                
                # sleep (prevent blocking)
                time.sleep(2)  
                              
            
        end_time = time.time()        
        str_log = f"\nClose File, Elapse: {round(end_time-start_time, 3)}"
        print(str_log)
        f.write(str_log)

        f.close()
        

In [22]:
df  = df[~df.client_id.isin(lab_transmitted_clients)]
l_clients             = list(set(df.client_id.values))
l_clients.sort()
print(f"Total clients: {len(l_clients)}")
SENDER_1 = l_clients[:50]

with open('sender_1_clients.txt', 'w') as fp:
    for client_id in SENDER_1:        
        fp.write(f"{client_id}\n")     

l_clients_to_send = []
with open('sender_1_clients.txt', 'r') as fp:
    for line in fp:
        l_clients_to_send.append(line.strip())        

Total clients: 132


In [23]:
play_clients(df, dataset_path,l_clients_to_send)


Start play beep, Elapse: 0.0

End play beep, Elapse: 1.058

Start play beep, Elapse: 1.108

End play beep, Elapse: 2.16

Start play beep, Elapse: 2.211

End play beep, Elapse: 3.263


Client_Id: 6387639b4d0f6689b78ce41f8ae1fa106b46c344fab45a21b8c562e244023635b091b1c8ffc6d54c6c6abfdd33a66903a36116f621ff79fea7b080df34bc1dae [1/50] Files: 11

Start play beep, Elapse: 3.315

End play beep, Elapse: 4.366

Start play beep, Elapse: 4.417

End play beep, Elapse: 5.469

Start play beep, Elapse: 5.625

End play beep, Elapse: 6.677

	[1/11] Start Play file: common_voice_ru_32637153.mp3, (length: 8.1 seconds), Elapse: 6.728



KeyboardInterrupt



<h1 style="background-color:#3cA8EF;"> <center><a id='parseLog'></a> Parse player log </center> </h1>

In [ ]:
def parse_log(log_file):
    
    arr_clients            = []   # arr of client_id [may be duplicate - depends on num of files per client]
    arr_cv                 = []   # cv file
    arr_time               = []   # cv file length    
    arr_num_cv_per_speaker = []   # for each speaker - number of speechs
    arr_start_speech_time  = []   # for each speech - start time
    arr_end_speech_time    = []

    with open(log_file) as f:
        lines = f.readlines()

        counter     = 0
        last_client = None
        start_time  = None        
        
        for line in lines:

            if line.find("Client_Id") != -1:
                
                client      = line[line.find(": ")+2 : line.find(" [")]
                last_client = client
                if counter > 0:
                    arr_num_cv_per_speaker.append(counter) 
                    counter = 0
                    
                 
            elif line.find("Start Play file:") != -1:                     
                start_time = float(line[line.find("Elapse:")+8:])                

                    
            elif line.find("---> Played file") != -1:                
                cv_file = line[line.find(": c")+2 : line.find(", ")]
                length  = float(line[line.find("length:")+8 : line.find(" seconds")])                
                end_t   = float(line[line.find("Elapse: ")+8:])                
                counter = counter + 1
                
                arr_clients.append(last_client)
                arr_cv.append(cv_file)
                arr_time.append(length)                
                arr_end_speech_time.append(end_t)
                if start_time is not None:
                    arr_start_speech_time.append(start_time)
                    start_time = None
                else:
                    print(f"Error, start_time is None")
           
            
    arr_num_cv_per_speaker.append(counter)         
    print(f"Total number of speakers: {len(set(arr_clients))}, files: {len(set(arr_cv))}")
    print(f"len(arr_start_speech_time) = {len(arr_start_speech_time)}")
    return arr_clients, arr_cv, arr_time,  arr_num_cv_per_speaker, arr_start_speech_time, arr_end_speech_time

In [ ]:
#NIGHT_RUN_1 = r"C:\Users\amitli\Repo\WalkieTalkieRecorder\log_01_04_2024_15_03_43.txt"
NIGHT_RUN_1 = r"/home/amitli/Repo/WalkieTalkieRecorder/log_08_05_2024_09_45_59.txt"
arr_clients, arr_cv, arr_time,  arr_num_cv_per_speaker, arr_start_speech_time, arr_end_speech_time = parse_log(NIGHT_RUN_1)

In [ ]:
import shutil



for cv_file in arr_cv:
    full_src_path = rf"{dataset_path}/clips/{cv_file}"
    full_dst_path = rf"/home/amitli/Repo/WalkieTalkieRecorder/Transmit_Files/{cv_file}"
    shutil.copyfile(full_src_path, full_dst_path)


<h1 style="background-color:#3cA8EF;"> <center> <a id='Reciever'></a> Reciever </center> </h1>

In [ ]:
from dataclasses        import dataclass, asdict
from concurrent.futures import ThreadPoolExecutor
from pydub              import AudioSegment,silence

import datetime
import pyaudio
import wave

In [ ]:

@dataclass
class StreamParams:
    format: int            = pyaudio.paInt16
    channels: int          = 1
    rate: int              = 8000
    frames_per_buffer: int = 1024
    input: bool            = True
    output: bool           = False

    def to_dict(self) -> dict:
        return asdict(self)

class Recorder:
    """Recorder uses the blocking I/O facility from pyaudio to record sound
    from mic.
    Attributes:
        - stream_params: StreamParams object with values for pyaudio Stream
            object
    """
    def __init__(self, stream_params: StreamParams) -> None:
        self.stream_params = stream_params
        self._pyaudio      = None
        self._stream       = None
        self._wav_file     = None
        self._counter      = 0

    def record(self, duration: int, save_path: str, num_files_to_create: int) -> None:
        """Record sound from mic for a given amount of seconds.
        :param duration: Number of seconds we want to record for
        :param save_path: Where to store recording
        """
        print("Start recording...")
        self.save_path = save_path
        self._create_recording_resources()
        self._write_wav_file_reading_from_stream(save_path, duration, num_files_to_create)
        self._close_recording_resources()
        print("Stop recording")

    def create_current_wav_file(self):
        
        self._counter = self._counter + 1 
        the_time  = datetime.datetime.now().strftime("%d_%m_%Y_%H_%M_%S%z")
        ctr_str   = f'{self._counter:04}'
        save_path = self.save_path.replace(".wav",f"_C_{ctr_str}_D_{the_time}.wav")        
        self._create_wav_file(save_path)
    
    def _create_recording_resources(self) -> None:
        self._pyaudio = pyaudio.PyAudio()
        self._stream = self._pyaudio.open(**self.stream_params.to_dict())
        self.create_current_wav_file()

    def _create_wav_file(self, save_path: str):
        print(f"creating new wav: {save_path}")
        self._wav_file = wave.open(save_path, "wb")
        self._wav_file.setnchannels(self.stream_params.channels)
        self._wav_file.setsampwidth(self._pyaudio.get_sample_size(self.stream_params.format))
        self._wav_file.setframerate(self.stream_params.rate)


    def close_current_wav_file(self, wav_file) -> None:
        wav_file.close()

    def _write_wav_file_reading_from_stream(self, save_path: str, duration: int, num_files_to_create: int) -> None:
        with ThreadPoolExecutor(max_workers = 5) as executor:
            for i in range(num_files_to_create):
                for _ in range(int(self.stream_params.rate * duration / self.stream_params.frames_per_buffer)):
                    audio_data = self._stream.read(self.stream_params.frames_per_buffer)
                    self._wav_file.writeframes(audio_data)
                    
                executor.submit(self.close_current_wav_file, self._wav_file)
                if i < num_files_to_create-1:
                    self.create_current_wav_file()

    def _close_recording_resources(self) -> None:
        self._stream.close()
        self._pyaudio.terminate()


In [ ]:
stream_params      = StreamParams()
stream_params.rate = 16000
recorder           = Recorder(stream_params)
recorder.record(60, "/home/amitli/Datasets/speakathon/audio.wav", num_files_to_create=60*18)

<h1 style="background-color:#3cA8EF;"> <center><a id='ParseRecords'></a>  Parse recordings </center> </h1>

In [ ]:
def get_sorted_files(files_path):
    all_files = glob.glob(files_path)
    all_files.sort()    
    return all_files    

In [ ]:
NIGHT_RUN_INPUT_FOLDER = r"/home/amitli/Debug/25_26_jul/Night_25_26/*"

In [ ]:
all_files = get_sorted_files(NIGHT_RUN_INPUT_FOLDER)
one_file  = AudioSegment.from_wav(all_files[0]) 
for i in range(1, len(all_files)):
    one_file = one_file + AudioSegment.from_wav(all_files[i])   

In [ ]:
NIGHT_RUN_1_ONE_FILE = r"/home/amitli/Debug/25_26_jul/One_file/One_file.wav"

In [ ]:
one_file.export(NIGHT_RUN_1_ONE_FILE, format="wav")

In [ ]:
arr_start_speech_time[:5]

In [ ]:
#AudioSegment.from_wav(r"/home/amitli/Debug/25_26_jul/Night_25_26/audio_C_0001_D_25_07_2023_15_18_31.wav")[9.9*1000:11*1000]

In [ ]:
full_wav              = AudioSegment.from_wav(NIGHT_RUN_1_ONE_FILE)
mp3_len               = arr_time
num_of_speakers       = len(set(arr_clients))
num_of_mp3_per_speker = arr_num_cv_per_speaker


start_rec_time_ms = int(9.9*1000) - arr_start_speech_time[0] * 1000
arr_results       = []
counter           = 0
end               = 0

for i_speaker in range(num_of_speakers):
   
    for i_mp3 in range(num_of_mp3_per_speker[i_speaker]):
                
        # --- speech
        start = start_rec_time_ms + arr_start_speech_time[counter]*1000
        #end   = arr_end_speech_time[counter]*1000 + start_rec_time_ms
        end   = start + mp3_len[counter]*1000   
        
        
        if end > len(full_wav):
            break
                   
        arr_results.append(full_wav[start-300 : end])                   
        counter = counter + 1
           

print(f"Finished at = {end} / {len(full_wav)}, Found: {len(arr_results)}")

In [ ]:
test = 2354
arr_results[test]

In [ ]:
print(f"{arr_cv[test]}, {arr_time[test]}, recorded: {arr_results[test].duration_seconds}")
AudioSegment.from_mp3(f"{ARABIC_PATH}/clips/{arr_cv[test]}")

In [ ]:
last_speaker  = 0
export_path   = "/home/amitli/Debug/25_26_jul/Outputs_wav"
full_path     = ""
for i in tqdm(range(2993)):
    
    speaker     = arr_clients[i]
    if speaker != last_speaker:
        expoer_folder = speaker
        full_path     = f"{export_path}/{speaker}"
        if not os.path.exists(full_path):  
            os.makedirs(full_path)        
        
    last_speaker = speaker
    tactic_file  = arr_results[i]
    cv_name      = arr_cv[i]
    lang         = arr_lang[i]
    
    file_name    = f"tactic_{lang}_{cv_name}"
    #arr_results[i].export(f"{full_path}/{file_name}", format="mp3")
    arr_results[i].export(f"{full_path}/{file_name[:-4]}.wav", format="wav")    

<h1 style="background-color:#3cA8EF;"> <center> <a id='Plot_dBFS'></a>  Plot (dBFS) </center> </h1>

In [ ]:
import plotly.graph_objects as go
import numpy as np
import wave

def read_wav_file(file_path):
    with wave.open(file_path, "rb") as wav_file:
        num_frames   = wav_file.getnframes()
        sample_width = wav_file.getsampwidth()
        sample_rate  = wav_file.getframerate()
        audio_data   = np.frombuffer(wav_file.readframes(num_frames), dtype=np.int16) 
    return audio_data, sample_rate


def convert_to_dbfs(audio_data):
    max_value = np.max(np.abs(audio_data))
    dbfs = 20 * np.log10(audio_data.astype(np.float32) / max_value)
    return dbfs


def plot_dBFS(audio_data, sample_rate):
    duration = len(audio_data) / sample_rate
    time = np.linspace(0, duration, len(audio_data))
    dbfs = convert_to_dbfs(audio_data)

    fig = go.Figure()
    fig.add_trace(go.Scatter(x=time, y=dbfs, mode="lines"))
    fig.update_layout(
        title="dBFS Plot",
        xaxis_title="Time (s)",
        yaxis_title="dBFS",
    )
    return fig

    
file_path = r"/home/amitli/Debug/25_26_jul/Night_25_26/audio_C_0001_D_25_07_2023_15_18_31.wav"
audio_data, sample_rate = read_wav_file(file_path)
fig                     = plot_dBFS(audio_data, sample_rate)
fig.show()    

[Go to start](#start_cell) </br>
[Parse log](#parseLog) </br>
